# The Kronecker theta model with 5 modes

In [1]:
from sympy import *
(x, y, X, Y, Z, B, C, n, m, l, j, q, N, M, K, k, z, s, p1, p2, p3, g2, g3,
 z0, w1, w2, w3, e1, e2, e3, Delta, 
 omega1, omega2, omega3, eta1, eta2, eta3, d1, d2, d3) = symbols(
    '''x, y, X, Y, Z, B, C, n, m, l, j, q, N, M, K, k, z, s, p1, p2, p3, g2, g3,
    z0, w1, w2, w3, e1, e2, e3, Delta, 
    omega1, omega2, omega3, eta1, eta2, eta3, d1, d2, d3'''
)
alpha, delta, t, nu, theta = symbols('alpha, delta, t, nu, theta')
pw = Function('pw') # Weierstrass P function
pwp = Function('pwp') # Derivative of Weierstrass P function
zw = Function('zw') # Weierstrass Zeta function
sigma = Function('sigma') # Weierstrass Sigma function
rho = Function('rho')
f = Function('f')
h = Function('h')
s = Function('s')
P = Function('P') # Polynomial
Q = Function('Q') # Polynomial
phi = Function('phi')
Phi = Function('Phi')
phi1 = Function('phi1')
phi2 = Function('phi2')
phi3 = Function('phi3')
A = IndexedBase('A')
W1 = Function('W1')
W2 = Function('W2')
W3 = Function('W3')
W4 = Function('W4')
W5 = Function('W5')
W6 = Function('W6')
Det = Function("Det")

kappa = IndexedBase('kappa')
beta = IndexedBase('beta')
mu = IndexedBase('mu')
xi = IndexedBase('xi')
a = IndexedBase('a')
b = IndexedBase('b')
c = IndexedBase('c')
d = IndexedBase('d')
p = IndexedBase('p')
F = IndexedBase('F')
G = IndexedBase('G')
psi = IndexedBase('psi')
upsilon = IndexedBase('upsilon')
epsilon = IndexedBase('epsilon')
WPdet = Function('WPdet')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# kth order derivatives of Weierstrass P
from wpk import wpk, run_tests

# The package containing mpmath expressions for Weierstrass elliptic functions
from numerical_evaluation.weierstrass_modified import Weierstrass
we = Weierstrass()
from mpmath import exp as mpexp

def mpc_to_float(mpc_val):
    return float(mpc_val.real) + float(mpc_val.imag)*1j

In [2]:
sigma_product_f_s = (
    (-1)**(N*(N-1)/2)*
    Product(factorial(k),(k,1,N))*
    sigma(Sum(mu[j],(j,0,N)),g2,g3)*
    Product(Product(Piecewise((sigma(mu[l] - mu[m], g2, g3),l<m),(1,True)),(l,0,N)),(m,0,N))/
    Product(sigma(mu[n],g2,g3)**(N+1),(n,0,N))
)

def frob_stick(Nval, evaluated=False, WPversion=False):
    """
    Frobenius Stickelberger in its original form
    """
    if evaluated:
        return Eq(p_matrix_f_s(Nval).det(), sigma_product_f_s.subs(N,Nval).doit())
    elif WPversion:
        return Eq(WPdet(Nval), sigma_product_f_s.subs(N,Nval).doit())
    return Eq(Det(p_matrix_f_s(Nval)), sigma_product_f_s.subs(N,Nval).doit())

In [3]:
sigma_product_f_s_N_factored = (
    (-1)**(N*(N-1)/2)*
    Product(factorial(k),(k,1,N-1))*
    factorial(N)*
    sigma(Sum(mu[j],(j,0,N)),g2,g3)*
    Product(Product(Piecewise((sigma(mu[l] - mu[m], g2, g3),l<m),(1,True)),(l,0,N-1)),(m,0,N-1))*
    Product(Piecewise((sigma(mu[l] - mu[N], g2, g3),l<N),(1,True)),(l,0,N))/
    Product(sigma(mu[n],g2,g3)**(N),(n,0,N))/
    Product(sigma(mu[n],g2,g3),(n,0,N))
)

sigma_product_f_s_N_over_N_minus_1 = (sigma_product_f_s_N_factored/sigma_product_f_s.subs(N,N-1))

subs1_list = [
    (Product(Piecewise((sigma(mu[l] - mu[N], g2, g3),l<N),(1,True)),(l,0,N)), 
     (-1)**N*Product(sigma(z - xi[l], g2, g3),(l,0,N))/sigma(z - xi[N], g2, g3)),
    (Product(sigma(mu[n],g2,g3),(n,0,N)),Product(sigma(xi[n],g2,g3),(n,0,N))*sigma(z,g2,g3)/sigma(xi[N],g2,g3)),
    (sigma(Sum(mu[j],(j,0,N-1)),g2,g3),-sigma(xi[N],g2,g3)),
    (Sum(mu[j],(j,0,N)),-xi[N]+z),
    (sigma(mu[N],g2,g3),sigma(z,g2,g3))
]
subs2_list = [
    ((-1)**(2*N)*Product(sigma(z - xi[l], g2, g3),(l,0,N)), 
   Product(sigma(z - xi[l], g2, g3)/sigma(xi[l], g2, g3)/sigma(z, g2, g3),(l,0,N)) *
   Product(sigma(xi[n], g2, g3),(n,0,N)) *
   (Product(sigma(z, g2, g3),(l,0,N)).doit()))
]

wp_sigma_product_fs_N_over_N_minus_1_mu_n = Eq(WPdet(N)/WPdet(N-1), sigma_product_f_s_N_over_N_minus_1).subs(
    Product(sigma(mu[n],g2,g3)**N,(n,0,N)),sigma(mu[N],g2,g3)**N*Product(sigma(mu[n],g2,g3)**N,(n,0,N-1))
).expand()

wp_sigma_product_fs_N_over_N_minus_1 = (
    wp_sigma_product_fs_N_over_N_minus_1_mu_n
    .subs(subs1_list)
    .subs(subs2_list)
).expand()
sigma_product_f_s_N_over_N_minus_1 = wp_sigma_product_fs_N_over_N_minus_1.rhs


def p_matrix_f_s(N): 
    return Matrix([[1, *[pw(x,g2,g3).diff((x,j)).subs(x,mu[k]) for j in range(N)]] for k in range(N+1)])

def p_matrix_f_s_1dz(N): 
    return Matrix([*[[1, *[pw(x,g2,g3).diff((x,j)).subs(x,mu[k]) for j in range(N)]] for k in range(N)],
       [1, *[pw(z,g2,g3).diff((z,j)) for j in range(N)]]])

def frob_stick_1dz_ratio(Nval, evaluated=False):
    """
    Frobenius Stickelberger adapted for Kronecker theta functions
    """
    if evaluated:
        return Eq(
            (((-1)**(Nval-1)/factorial(Nval) * p_matrix_f_s_1dz(Nval).det()/p_matrix_f_s(Nval-1).det()).expand())
                  .collect([pw(z,g2,g3), diff(pw(z,g2,g3),z)],factor), 
            (-1)**(Nval-1)/factorial(Nval) * sigma_product_f_s_N_over_N_minus_1.subs(N,Nval).doit().subs(
            xi[Nval],-Sum(xi[j],(j,0,Nval-1)).doit()
        ).subs([(xi[j],mu[j]) for j in range(Nval)]))
    
    return Eq(
        (-1)**(Nval-1)/factorial(Nval) * Det(p_matrix_f_s_1dz(Nval))/Det(p_matrix_f_s(Nval-1)), 
        
        (-1)**(Nval-1)/factorial(Nval) * sigma_product_f_s_N_over_N_minus_1.subs(N,Nval).doit().subs(
            xi[Nval],-Sum(xi[j],(j,0,Nval-1)).doit()
        ).subs([(xi[j],mu[j]) for j in range(Nval)]))

## The case with 5 modes

In [4]:
unevaluated_4 = frob_stick_1dz_ratio(Nval=4)
unevaluated_4

Eq(-Det(Matrix([
[1, pw(mu[0], g2, g3), Derivative(pw(mu[0], g2, g3), mu[0]), Derivative(pw(mu[0], g2, g3), (mu[0], 2)), Derivative(pw(mu[0], g2, g3), (mu[0], 3))],
[1, pw(mu[1], g2, g3), Derivative(pw(mu[1], g2, g3), mu[1]), Derivative(pw(mu[1], g2, g3), (mu[1], 2)), Derivative(pw(mu[1], g2, g3), (mu[1], 3))],
[1, pw(mu[2], g2, g3), Derivative(pw(mu[2], g2, g3), mu[2]), Derivative(pw(mu[2], g2, g3), (mu[2], 2)), Derivative(pw(mu[2], g2, g3), (mu[2], 3))],
[1, pw(mu[3], g2, g3), Derivative(pw(mu[3], g2, g3), mu[3]), Derivative(pw(mu[3], g2, g3), (mu[3], 2)), Derivative(pw(mu[3], g2, g3), (mu[3], 3))],
[1,     pw(z, g2, g3),         Derivative(pw(z, g2, g3), z),         Derivative(pw(z, g2, g3), (z, 2)),         Derivative(pw(z, g2, g3), (z, 3))]]))*Det(Matrix([
[1, pw(mu[0], g2, g3), Derivative(pw(mu[0], g2, g3), mu[0]), Derivative(pw(mu[0], g2, g3), (mu[0], 2))],
[1, pw(mu[1], g2, g3), Derivative(pw(mu[1], g2, g3), mu[1]), Derivative(pw(mu[1], g2, g3), (mu[1], 2))],
[1, pw(mu[2], g2, 

In [5]:
vals = [mu[_k] for _k in range(4)]
vals.append(z)
wpk_subs = [wpk(_n).subs(z, val) for val in vals for _n in range(1,4)][::-1]

In [6]:
evaluated_4 = frob_stick_1dz_ratio(Nval=4, evaluated=True)

In [7]:
evaluated_4_with_subs = evaluated_4.subs([_eq.args for _eq in wpk_subs])

In [8]:
__x1234 = Eq(-12*Det(Matrix([
 [1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2, pw(mu[0], g2, g3)*pwp(mu[0], g2, g3)],
 [1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2, pw(mu[1], g2, g3)*pwp(mu[1], g2, g3)],
 [1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2, pw(mu[2], g2, g3)*pwp(mu[2], g2, g3)],
 [1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2, pw(mu[3], g2, g3)*pwp(mu[3], g2, g3)],
 [1,     pw(z, g2, g3),     pwp(z, g2, g3), pw(z, g2, g3)**2, pw(z, g2, g3)*pwp(z, g2, g3)]]))*Det(Matrix([
 [1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2],
 [1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2],
 [1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2],
 [1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2]]))**(-1)/24,
 -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*
             sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*
             sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/
             (sigma(z, g2, g3)**5*
              sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*
              sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3))
)

__x1234b = Eq(-12*Det(Matrix([
 [1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2 - 2*pw(mu[0], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2, pw(mu[0], g2, g3)*pwp(mu[0], g2, g3)],
 [1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2- 2*pw(mu[1], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2, pw(mu[1], g2, g3)*pwp(mu[1], g2, g3)],
 [1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2- 2*pw(mu[2], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2, pw(mu[2], g2, g3)*pwp(mu[2], g2, g3)],
 [1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2- 2*pw(mu[3], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2, pw(mu[3], g2, g3)*pwp(mu[3], g2, g3)],
 [1,     pw(z, g2, g3),     pwp(z, g2, g3), 0, pw(z, g2, g3)*pwp(z, g2, g3)]]))*Det(Matrix([
 [1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2],
 [1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2],
 [1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2],
 [1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2]]))**(-1)/24,
 -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*
             sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*
             sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/
             (sigma(z, g2, g3)**5*
              sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*
              sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3))
)

__x1234c = Eq(-12*Det(Matrix([
 [1, pw(mu[0], g2, g3) - pw(z, g2, g3), pwp(mu[0], g2, g3), factor(pw(mu[0], g2, g3)**2 - 2*pw(mu[0], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2), factor(pw(mu[0], g2, g3)*pwp(mu[0], g2, g3) - pw(z, g2, g3)*pwp(mu[0], g2, g3))],
 [1, pw(mu[1], g2, g3) - pw(z, g2, g3), pwp(mu[1], g2, g3), factor(pw(mu[1], g2, g3)**2- 2*pw(mu[1], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2), factor(pw(mu[1], g2, g3)*pwp(mu[1], g2, g3) - pw(z, g2, g3)*pwp(mu[1], g2, g3))],
 [1, pw(mu[2], g2, g3) - pw(z, g2, g3), pwp(mu[2], g2, g3), factor(pw(mu[2], g2, g3)**2- 2*pw(mu[2], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2), factor(pw(mu[2], g2, g3)*pwp(mu[2], g2, g3) - pw(z, g2, g3)*pwp(mu[2], g2, g3))],
 [1, pw(mu[3], g2, g3) - pw(z, g2, g3), pwp(mu[3], g2, g3), factor(pw(mu[3], g2, g3)**2- 2*pw(mu[3], g2, g3)*pw(z, g2, g3) + pw(z, g2, g3)**2), factor(pw(mu[3], g2, g3)*pwp(mu[3], g2, g3) - pw(z, g2, g3)*pwp(mu[3], g2, g3))],
 [1,     0,     pwp(z, g2, g3), 0, 0]]))*Det(Matrix([
 [1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2],
 [1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2],
 [1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2],
 [1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2]]))**(-1)/24,
 -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*
             sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*
             sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/
             (sigma(z, g2, g3)**5*
              sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*
              sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3))
)

__x1234c

Eq(-Det(Matrix([
[1, -pw(z, g2, g3) + pw(mu[0], g2, g3), pwp(mu[0], g2, g3), (-pw(z, g2, g3) + pw(mu[0], g2, g3))**2, (-pw(z, g2, g3) + pw(mu[0], g2, g3))*pwp(mu[0], g2, g3)],
[1, -pw(z, g2, g3) + pw(mu[1], g2, g3), pwp(mu[1], g2, g3), (-pw(z, g2, g3) + pw(mu[1], g2, g3))**2, (-pw(z, g2, g3) + pw(mu[1], g2, g3))*pwp(mu[1], g2, g3)],
[1, -pw(z, g2, g3) + pw(mu[2], g2, g3), pwp(mu[2], g2, g3), (-pw(z, g2, g3) + pw(mu[2], g2, g3))**2, (-pw(z, g2, g3) + pw(mu[2], g2, g3))*pwp(mu[2], g2, g3)],
[1, -pw(z, g2, g3) + pw(mu[3], g2, g3), pwp(mu[3], g2, g3), (-pw(z, g2, g3) + pw(mu[3], g2, g3))**2, (-pw(z, g2, g3) + pw(mu[3], g2, g3))*pwp(mu[3], g2, g3)],
[1,                                  0,     pwp(z, g2, g3),                                       0,                                                       0]]))*Det(Matrix([
[1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2],
[1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2],
[1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3)

In [9]:
Fk_eq = Eq(pw(z,g2,g3) - pw(mu[k],g2,g3), F[k])
Fk_eq

Eq(pw(z, g2, g3) - pw(mu[k], g2, g3), F[k])

In [10]:
(Fk_eq.lhs, Fk_eq.rhs)

(pw(z, g2, g3) - pw(mu[k], g2, g3), F[k])

In [11]:
C_det = Eq(Det(Matrix([
 [1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2],
 [1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2],
 [1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2],
 [1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2]])), C)

__x1234d = __x1234c.subs([Fk_eq.subs(k,_k).args for _k in range(4)]).subs(*C_det.args)


C_det
__x1234d

Eq(Det(Matrix([
[1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2],
[1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2],
[1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2],
[1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2]])), C)

Eq(-Det(Matrix([
[1, -F[0], pwp(mu[0], g2, g3), F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
[1, -F[1], pwp(mu[1], g2, g3), F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
[1, -F[2], pwp(mu[2], g2, g3), F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
[1, -F[3], pwp(mu[3], g2, g3), F[3]**2, -pwp(mu[3], g2, g3)*F[3]],
[1,     0,     pwp(z, g2, g3),       0,                        0]]))/(2*C), -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)))

In [12]:
__x1234d3 = Eq(-Det(Matrix([
 [-F[0], pwp(mu[0], g2, g3) - pwp(z, g2, g3), F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
 [-F[1], pwp(mu[1], g2, g3) - pwp(z, g2, g3), F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
 [-F[2], pwp(mu[2], g2, g3) - pwp(z, g2, g3), F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
 [-F[3], pwp(mu[3], g2, g3) - pwp(z, g2, g3), F[3]**2, -pwp(mu[3], g2, g3)*F[3]]]))/(2*C),
 -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*
               sigma(z - mu[2], g2, g3)*
               sigma(z - mu[3], g2, g3)*
               sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/
               (sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)
                *sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3))
)

__x1234d3

Eq(-Det(Matrix([
[-F[0], -pwp(z, g2, g3) + pwp(mu[0], g2, g3), F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
[-F[1], -pwp(z, g2, g3) + pwp(mu[1], g2, g3), F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
[-F[2], -pwp(z, g2, g3) + pwp(mu[2], g2, g3), F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
[-F[3], -pwp(z, g2, g3) + pwp(mu[3], g2, g3), F[3]**2, -pwp(mu[3], g2, g3)*F[3]]]))/(2*C), -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)))

In [13]:
__x1234e = Eq(-Det(Matrix([
 [-F[0], pwp(mu[0], g2, g3), F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
 [-F[1], pwp(mu[1], g2, g3), F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
 [-F[2], pwp(mu[2], g2, g3), F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
 [-F[3], pwp(mu[3], g2, g3), F[3]**2, -pwp(mu[3], g2, g3)*F[3]]]))/(2*C)
              -pwp(z, g2, g3)*Det(Matrix([
 [1, -F[0], F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
 [1, -F[1], F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
 [1, -F[2], F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
 [1, -F[3], F[3]**2, -pwp(mu[3], g2, g3)*F[3]]]))/(2*C),
 -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*
              sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*
              sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/
              (sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*
               sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3))
)

__x1234e

Eq(-pwp(z, g2, g3)*Det(Matrix([
[1, -F[0], F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
[1, -F[1], F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
[1, -F[2], F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
[1, -F[3], F[3]**2, -pwp(mu[3], g2, g3)*F[3]]]))/(2*C) - Det(Matrix([
[-F[0], pwp(mu[0], g2, g3), F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
[-F[1], pwp(mu[1], g2, g3), F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
[-F[2], pwp(mu[2], g2, g3), F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
[-F[3], pwp(mu[3], g2, g3), F[3]**2, -pwp(mu[3], g2, g3)*F[3]]]))/(2*C), -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)))

In [14]:
h_5modes_eq_a = Eq(__x1234e.lhs + __x1234e.lhs.subs(z,-z), 
                 __x1234e.rhs + __x1234e.rhs.subs(z,-z)).subs(pwp(-z,g2,g3), -pwp(z,g2,g3))

h_5modes_eq_b = Eq(
    h_5modes_eq_a.lhs.replace(Det,det)
    .collect(
        ((Sum(pwp(mu[k],g2,g3),(k,0,3))**2 - Sum(pwp(mu[k],g2,g3)**2,(k,0,3)))/2).doit().expand().args, 
        factor
    ),
    h_5modes_eq_a.rhs
)

h_5modes_eq_a
h_5modes_eq_b

Eq(-Det(Matrix([
[-F[0], pwp(mu[0], g2, g3), F[0]**2, -pwp(mu[0], g2, g3)*F[0]],
[-F[1], pwp(mu[1], g2, g3), F[1]**2, -pwp(mu[1], g2, g3)*F[1]],
[-F[2], pwp(mu[2], g2, g3), F[2]**2, -pwp(mu[2], g2, g3)*F[2]],
[-F[3], pwp(mu[3], g2, g3), F[3]**2, -pwp(mu[3], g2, g3)*F[3]]]))/C, -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)) - sigma(-z - mu[0], g2, g3)*sigma(-z - mu[1], g2, g3)*sigma(-z - mu[2], g2, g3)*sigma(-z - mu[3], g2, g3)*sigma(-z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(-z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)))

Eq(((F[0] - F[1])*(F[2] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[1], g2, g3)*F[2]*F[3] + (F[0] - F[1])*(F[2] - F[3])*pwp(mu[2], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[1] - (F[0] - F[2])*(F[1] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[2], g2, g3)*F[1]*F[3] - (F[0] - F[2])*(F[1] - F[3])*pwp(mu[1], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[2] + (F[0] - F[3])*(F[1] - F[2])*pwp(mu[0], g2, g3)*pwp(mu[3], g2, g3)*F[1]*F[2] + (F[0] - F[3])*(F[1] - F[2])*pwp(mu[1], g2, g3)*pwp(mu[2], g2, g3)*F[0]*F[3])/C, -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)) - sigma(-z - mu[0], g2, g3)*sigma(-z - mu[1], g2, g3)*sigma(-z - mu[2], g2, g3)*sigma(-z - mu[3], g2, g3)*sigma(-z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(-z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*

In [15]:
((Sum(pwp(mu[k],g2,g3),(k,0,3))**2 - Sum(pwp(mu[k],g2,g3)**2,(k,0,3)))/2).doit().expand().args

(pwp(mu[0], g2, g3)*pwp(mu[1], g2, g3),
 pwp(mu[0], g2, g3)*pwp(mu[2], g2, g3),
 pwp(mu[0], g2, g3)*pwp(mu[3], g2, g3),
 pwp(mu[1], g2, g3)*pwp(mu[2], g2, g3),
 pwp(mu[1], g2, g3)*pwp(mu[3], g2, g3),
 pwp(mu[2], g2, g3)*pwp(mu[3], g2, g3))

In [16]:
C_det

Eq(Det(Matrix([
[1, pw(mu[0], g2, g3), pwp(mu[0], g2, g3), pw(mu[0], g2, g3)**2],
[1, pw(mu[1], g2, g3), pwp(mu[1], g2, g3), pw(mu[1], g2, g3)**2],
[1, pw(mu[2], g2, g3), pwp(mu[2], g2, g3), pw(mu[2], g2, g3)**2],
[1, pw(mu[3], g2, g3), pwp(mu[3], g2, g3), pw(mu[3], g2, g3)**2]])), C)

### Numeric tests

In [17]:
from random import random
g2val_ = (random() - 0.5) + (random() - 0.5)*1j
g3val_ = (random() - 0.5) + (random() - 0.5)*1j
zval_ = (random() - 0.5) + (random() - 0.5)*1j
mu_vals = [(random() - 0.5) + (random() - 0.5)*1j for _ in range(4)]

omega_ = we.omega_from_g(g2val_, g3val_)
omega1_ = mpc_to_float(omega_[0])
omega2_ = mpc_to_float(omega_[1])
omega3_ = mpc_to_float(omega_[2])
if im(omega2_/omega1_) <= 0:
    omega2_ = -omega2_
omegas = (omega1_, omega2_)  

num_subs = (
    [(pw(mu[k],g2,g3), we.wp(mu_vals[k], omegas)) for k in range(len(mu_vals))] + 
    [(pwp(mu[k],g2,g3), we.wpprime(mu_vals[k], omegas)) for k in range(len(mu_vals))] + 
    [(pw(z,g2,g3), we.wp(zval_, omegas)), (pwp(z,g2,g3), we.wpprime(zval_, omegas))] +
    [(sigma(z - mu[k], g2, g3), we.wsigma(zval_ - mu_vals[k], omegas)) for k in range(len(mu_vals))] + 
    [(sigma(mu[k], g2, g3), we.wsigma(mu_vals[k], omegas)) for k in range(len(mu_vals))] +
    [(sigma(z , g2, g3), we.wsigma(zval_ , omegas))] + 
    [(sigma(z - sum([mu[k] for k in range(len(mu_vals))]), g2, g3), we.wsigma(zval_ - sum(mu_vals), omegas))] + 
    [(sigma(z + sum([mu[k] for k in range(len(mu_vals))]), g2, g3), we.wsigma(zval_ + sum(mu_vals), omegas))] + 
    [(sigma(- sum([mu[k] for k in range(len(mu_vals))]), g2, g3), we.wsigma(- sum(mu_vals), omegas))]
)

In [18]:
evaluated_4_with_subs.lhs.simplify().subs(num_subs).expand().subs(g2, g2val_).evalf()

-7.40749383866979 - 27.5806438633621*I

In [19]:
evaluated_4_with_subs.rhs.simplify().subs(num_subs).expand().subs(g2, g2val_).evalf()

-7.40749383866964 - 27.5806438633621*I

In [20]:
__x1234c.lhs.simplify().subs(num_subs).expand().subs(g2, g2val_).replace(Det, det).evalf()

-7.40749383866966 - 27.5806438633622*I

In [21]:
__x1234c.rhs.simplify().subs(num_subs).expand().subs(g2, g2val_).replace(Det, det).evalf()

-7.40749383866964 - 27.5806438633621*I

## Lagrangian

In [22]:
h_5modes_eq_b

Eq(((F[0] - F[1])*(F[2] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[1], g2, g3)*F[2]*F[3] + (F[0] - F[1])*(F[2] - F[3])*pwp(mu[2], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[1] - (F[0] - F[2])*(F[1] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[2], g2, g3)*F[1]*F[3] - (F[0] - F[2])*(F[1] - F[3])*pwp(mu[1], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[2] + (F[0] - F[3])*(F[1] - F[2])*pwp(mu[0], g2, g3)*pwp(mu[3], g2, g3)*F[1]*F[2] + (F[0] - F[3])*(F[1] - F[2])*pwp(mu[1], g2, g3)*pwp(mu[2], g2, g3)*F[0]*F[3])/C, -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)) - sigma(-z - mu[0], g2, g3)*sigma(-z - mu[1], g2, g3)*sigma(-z - mu[2], g2, g3)*sigma(-z - mu[3], g2, g3)*sigma(-z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(-z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*

In [24]:
Eq(
    ((F[0] - F[1])*(F[2] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[1], g2, g3)*F[2]*F[3] + 
     (F[0] - F[1])*(F[2] - F[3])*pwp(mu[2], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[1] - 
     (F[0] - F[2])*(F[1] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[2], g2, g3)*F[1]*F[3] - 
     (F[0] - F[2])*(F[1] - F[3])*pwp(mu[1], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[2] + 
     (F[0] - F[3])*(F[1] - F[2])*pwp(mu[0], g2, g3)*pwp(mu[3], g2, g3)*F[1]*F[2] + 
     (F[0] - F[3])*(F[1] - F[2])*pwp(mu[1], g2, g3)*pwp(mu[2], g2, g3)*F[0]*F[3])/C,
 -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*
    sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/
    (sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)
     *sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)) - 
    sigma(-z - mu[0], g2, g3)*sigma(-z - mu[1], g2, g3)*sigma(-z - mu[2], g2, g3)*
    sigma(-z - mu[3], g2, g3)*sigma(-z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/
    (sigma(-z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)
     *sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3))
).subs()

Eq(((F[0] - F[1])*(F[2] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[1], g2, g3)*F[2]*F[3] + (F[0] - F[1])*(F[2] - F[3])*pwp(mu[2], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[1] - (F[0] - F[2])*(F[1] - F[3])*pwp(mu[0], g2, g3)*pwp(mu[2], g2, g3)*F[1]*F[3] - (F[0] - F[2])*(F[1] - F[3])*pwp(mu[1], g2, g3)*pwp(mu[3], g2, g3)*F[0]*F[2] + (F[0] - F[3])*(F[1] - F[2])*pwp(mu[0], g2, g3)*pwp(mu[3], g2, g3)*F[1]*F[2] + (F[0] - F[3])*(F[1] - F[2])*pwp(mu[1], g2, g3)*pwp(mu[2], g2, g3)*F[0]*F[3])/C, -sigma(z - mu[0], g2, g3)*sigma(z - mu[1], g2, g3)*sigma(z - mu[2], g2, g3)*sigma(z - mu[3], g2, g3)*sigma(z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*sigma(mu[0], g2, g3)*sigma(mu[1], g2, g3)*sigma(mu[2], g2, g3)*sigma(mu[3], g2, g3)) - sigma(-z - mu[0], g2, g3)*sigma(-z - mu[1], g2, g3)*sigma(-z - mu[2], g2, g3)*sigma(-z - mu[3], g2, g3)*sigma(-z + mu[0] + mu[1] + mu[2] + mu[3], g2, g3)/(sigma(-z, g2, g3)**5*sigma(-mu[0] - mu[1] - mu[2] - mu[3], g2, g3)*

In [25]:
Fk_eq

Eq(pw(z, g2, g3) - pw(mu[k], g2, g3), F[k])